In [17]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [18]:
model_path = "../../corpus/sbert_stair2"
# data_name = "hate_labeled.csv"

In [19]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models
sbert = SentenceTransformer(model_path)

[24223] 2022-01-05 18:07:26,949 Info sentence_transformers.SentenceTransformer :Load pretrained SentenceTransformer: ../../corpus/sbert_stair2
[24223] 2022-01-05 18:07:27,699 Info sentence_transformers.SentenceTransformer :Use pytorch device: cuda


In [20]:
def text2vec(text):
    sentences = []
    if isinstance(text, str):
        sentences = [text]
    elif isinstance(text, list):
        sentences = text
    
    return sbert.encode(sentences)


In [21]:
la2la = {0:0,1:0, 2:1, 3:1}

In [22]:
def make_dataset(df, mode="All"):
    X = []
    y = []
    path = "../hand_labeled/"
    datalist = ['DCM', 'DIT', 'IRS']
    convs = read_conv(path, datalist)

    usr_utt = []
    for conv in convs:
        for i, ut in enumerate(conv):
            if not ut.is_system():
                usr_utt.append(clean_text(ut.utt))
    import random

    if mode=="All":
        for la, txt in zip(df.label, df.txt):
            # X.append( InputExample(guid="", texts=[txt], label=float(la) ) )
            X.append(txt)
            # y.append(la)
            y.append( la2la[la] )
        print("length of X", len(X))
        # 0 の要素を増やしておきますわよ
        sampled = random.sample(usr_utt, len(X)//3)
        for sample in sampled:
            # X.append( InputExample(guid="" , texts=[sample], label=0.0 ) )
            X.append(sample)
            y.append(0)
        print("added length of X", len(X))

    return X, y

In [23]:
corpus_path = "../../corpus/twitter/"
# data_name = "hate_labeled.csv"
data_name = "impolite.csv"
df = pd.read_csv(corpus_path+data_name)


In [24]:
max_ = 1100
df = df[:max_]

In [25]:
X, y = make_dataset(df)

length of X 1100
added length of X 1466


In [26]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [27]:
X_train = text2vec(X_train_str)

Batches:   0%|          | 0/33 [00:00<?, ?it/s]

In [28]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
# clf = AdaBoostClassifier()
clf = svm.SVC(kernel='rbf', gamma =0.0001, C=1000)

In [29]:
clf.fit(X_train, y_train)

SVC(C=1000, gamma=0.0001)

In [30]:
print(X_train_str[:5])
print(y_train[:5])

['が、こけた早苗ちゃんと同類のようにね!', '起きてるなら付き合え', 'いえいえ、こちらこそどういたしまして。', 'ウザ', 'おいも片付けていけや']
[1, 1, 0, 1, 1]


In [31]:
X_test = text2vec(X_test_str)

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

In [32]:
y_pred = clf.predict(X_test)

In [33]:
for y_p, x_s in zip(y_pred[:30], X_test_str[:30]):
    print("{0} : {1}".format(y_p, x_s))

0 : 最近俺で遊びすぎWW
1 : おみげのディスやべーやん
1 : まとめてぶっ飛ばす
1 : 寧ろ地獄を見るのは安倍晋三と与党の屑、ぢゃい
0 : へぇ初耳でした
1 : お前のせいだからな
1 : 調べるとフラウンダーを鰈とする所もあって、鰈と鮃区別つけてないやろってなってる
0 : まあ猫なんて、そんなもんですよね。
0 : はい、有名な所ではルパン3世の声ですね。
0 : どーしたの．同じくらい好きだよ
1 : 絶対半年に0回しか遊んでくれん
1 : ずるだったん知らんかった
0 : まだ寝るには早いですよ
1 : お前が落ち着きなぁ!
1 : お前が金出せよ。
1 : がんばれ
1 : なんて!幸せになれよ!クソ野郎!
1 : 私に怒って当たるな!!お前のせいだろうが!!
0 : そうなのですね。私も見たことがありません。
0 : ええて
1 : 盛大にやってんなW
0 : 私は饂飩です。
0 : そうなんですか。どんなキャンペーンだったんですか?
0 : 君の方が大事だから死ぬなよ…
1 : 巻き込むなよ
0 : 食えない俺より、一々そんな反応するの貧乏臭さの方が滑稽だわ
1 : 夜もバロすんの?
1 : お前のせいなんだぞッ!!
1 : プリ撮ったんなら見せろ
1 : 落ち着けって


In [34]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[185  30]
 [ 34 191]]
accuracy =  0.8545454545454545


In [41]:
import copy
def convert_bool(p):
    new_p = copy.deepcopy(p)
    for i, v in enumerate(p):
        if v<=1:
            new_p[i] = 0
        else:
            new_p[i] = 1
    return new_p

def convert_bool2(p):
    new_p = copy.deepcopy(p)
    return new_p

In [42]:
y_test_ = convert_bool2(y_test)
y_pred_ = convert_bool2(y_pred)

In [43]:
print('confusion matrix = \n', confusion_matrix(y_true=y_test_, y_pred=y_pred_))
print('accuracy = ', accuracy_score(y_true=y_test_, y_pred=y_pred_))
print('precision = ', precision_score(y_true=y_test_, y_pred=y_pred_))
print('recall = ', recall_score(y_true=y_test_, y_pred=y_pred_))
print('f1 score = ', f1_score(y_true=y_test_, y_pred=y_pred_))

confusion matrix = 
 [[185  30]
 [ 34 191]]
accuracy =  0.8545454545454545
precision =  0.8642533936651584
recall =  0.8488888888888889
f1 score =  0.8565022421524665


In [38]:
tests = ["知らんのかい", "お前馬鹿だろ", "センスないね", "雑魚がよ", "俺が馬鹿だった","馬鹿みたいに笑ったw", "消えろカス", "お前頭が悪いな", "あの国のやつらはクズ"]

In [44]:
vec = text2vec(tests)
test_res = convert_bool2( clf.predict(vec) ) 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [45]:
for y_p, x_s in zip(test_res, tests):
    print("{0} : {1}".format(y_p, x_s))

1 : 知らんのかい
1 : お前馬鹿だろ
1 : センスないね
1 : 雑魚がよ
1 : 俺が馬鹿だった
0 : 馬鹿みたいに笑ったw
1 : 消えろカス
1 : お前頭が悪いな
1 : あの国のやつらはクズ


In [ ]:
# ここから学習データでテスト

In [52]:
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [53]:

error = "Lack of sociality"
# errors = ['Grammatical error', "Uninterpretable"]
sys_utt = []
y_ = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
            # if not ut.utt[-1] in ["？", "！", "。", "!"]:
            #     # sys_utt.append(ut.utt+"。")
            #     sys_utt.append(ut.utt)
            # else:   
            sys_utt.append(ut.utt)
            if ut.is_error_included(error):
                y_.append(1)
                # print(ut.utt)
            else:
                y_.append(0)


In [54]:
print("len of y:{0}, error '{1}' counts:{2}".format(len(y_), error, y_.count(1)))

len of y:1386, error 'Lack of sociality' counts:7


In [55]:
vec = text2vec(sys_utt)
y_pred_ = convert_bool2( clf.predict(vec) ) 

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

In [56]:
print('confusion matrix = \n', confusion_matrix(y_true=y_, y_pred=y_pred_))
print('accuracy = ', accuracy_score(y_true=y_, y_pred=y_pred_))
print('precision = ', precision_score(y_true=y_, y_pred=y_pred_))
print('recall = ', recall_score(y_true=y_, y_pred=y_pred_))
print('f1 score = ', f1_score(y_true=y_, y_pred=y_pred_))

confusion matrix = 
 [[1313   66]
 [   3    4]]
accuracy =  0.9502164502164502
precision =  0.05714285714285714
recall =  0.5714285714285714
f1 score =  0.10389610389610389


In [57]:
# for utt, a, b in zip(sys_utt, y_,  y_pred_):
#     if a==0 and b==1:
#         print(utt)

- 検出は不可能でしょこれは

        confusion matrix = 
        [[1314   34]
        [   1    0]]
        accuracy =  0.9740548554484804
        precision =  0.0
        recall =  0.0
        f1 score =  0.0